In [1]:
import tensorflow as tf
import keras
import random
import gym
import numpy as np
from collections import deque
from keras.models import Sequential
from keras.layers import Dense
from keras import initializers
from keras.optimizers import Adam
import json
from keras.models import model_from_json
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.optimizers import SGD , Adam
import tensorflow as tf
import skimage
from skimage import color, exposure, transform

Using TensorFlow backend.


In [2]:
env = gym.make('PongDeterministic-v4')

[2017-12-23 18:25:32,328] Making new env: PongDeterministic-v4


In [3]:
ACTIONS = env.action_space.n # number of valid actions
GAMMA = 0.99 # decay rate of past observations
OBSERVATION = 500. # timesteps to observe before training
EXPLORE = 1000000. # frames over which to anneal epsilon
FINAL_EPSILON = 0.01 # final value of epsilon
INITIAL_EPSILON = 0.4 # starting value of epsilon
REPLAY_MEMORY = 20000 # number of previous transitions to remember
BATCH = 64 # size of minibatch
LEARNING_RATE = 1e-4
SGD_LEARNING_RATE = 1e-1

img_rows , img_cols = 84, 84
#Convert image into Black and white
img_channels = 3 #We stack 3 frames

max_epLength = 200

NUM_EPISODES = 2000

SAVE_DIR = 'dqn_pong/'

In [4]:
import os
if not os.path.exists(SAVE_DIR):
    os.makedirs(SAVE_DIR)

In [5]:
import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

In [6]:
def build_model():
    
    model = Sequential()
    model.add(Conv2D(32, (8, 8), strides=(4, 4), padding='valid',input_shape=(img_rows,img_cols,img_channels)))  #80*80*4
    model.add(Activation('relu'))
    model.add(Conv2D(64, (4, 4), strides=(2, 2), padding='valid'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3), strides=(1, 1), padding='valid'))
    model.add(Activation('relu'))
    #model.add(Activation('tanh'))
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dense(ACTIONS, activation='linear'))

    #adam = Adam(lr=LEARNING_RATE)
    #model.compile(loss='mse',optimizer=adam)
    sgd = SGD(lr=SGD_LEARNING_RATE)
    model.compile(loss='mse',optimizer=sgd)
    print("We finish building the model")
    return model

In [7]:
from skimage import data, io
from matplotlib import pyplot as plt
    
def show_as_img(arr):
    io.imshow(arr.reshape(img_rows, img_cols, 3))
    plt.show()

In [8]:
class Memory():
    def __init__(self, buff_sz):
        self.buff_sz = buff_sz
        self.M = deque()
    def append(self, tup):
        self.M.append(tup)
        if (len(self.M) > self.buff_sz):
            dump = self.M.popleft()
            if dump[2] != 0.0:
                if random.random() < 0.5:
                    self.append(dump)
    def sample(self, num_samples):
        minibatch = random.sample(self.M, num_samples)
        return minibatch
        #indices_random = random.randrange(0, len(self.M) - num_samples)
        #return list(self.M)[indices_random:indices_random + num_samples]

In [9]:
def save_model(model, path):
    model.save(path)

In [10]:
def process_frame(x_t):
    x_t = skimage.color.rgb2gray(x_t)
    x_t = skimage.transform.resize(x_t,(img_cols,img_rows), mode='constant')
    x_t = skimage.exposure.rescale_intensity(x_t,out_range=(0,255))
    x_t = x_t.reshape((1, img_cols, img_rows, 1))
    x_t /= 255.0
    return x_t

In [11]:
import time

TIMESTEP = 0

def train_model(model, env):
    
    M = Memory(REPLAY_MEMORY)
 
    OBSERVE = OBSERVATION
    epsilon = INITIAL_EPSILON

    t = 0
    
    rewards = []
    
    for idxEpisode in range(NUM_EPISODES):
        #Reset environment and get first new observation
        x_t = env.reset()
        x_t = process_frame(x_t)
        s_t = np.stack((x_t, x_t, x_t), axis=3)
        s_t = s_t.reshape(1, s_t.shape[1], s_t.shape[2], s_t.shape[3])
        d = False
        rAll = 0
        j = 0
        loss = 0.0
        ct_non_zero_reward = 0
        #The Q-Network
        while j < max_epLength: #If the agent takes longer than 200 moves to reach either of the blocks, end the trial.
            j+=1
            a_t = None
            #Choose an action by greedily (with e chance of random action) from the Q-network
            if np.random.rand(1) < epsilon or t < OBSERVE:
                a_t = random.randrange(ACTIONS)
            else:
                q = model.predict(s_t)
                policy_max_Q = np.argmax(q)
                a_t = policy_max_Q
            x_t1,r_t,done,_ = env.step(a_t)
            x_t1 = process_frame(x_t1)
            s_t1 = np.append(x_t1, s_t[:, :, :, :2], axis=3)
            
            t += 1
            TIMESTEP = t
            M.append((s_t, a_t, r_t, s_t1, done))
            
            if epsilon > FINAL_EPSILON and t > OBSERVE:
                epsilon -= (INITIAL_EPSILON - FINAL_EPSILON) / EXPLORE
                minibatch = M.sample(BATCH)
                inputs = np.zeros((BATCH, s_t.shape[1], s_t.shape[2], s_t.shape[3]))
                targets = np.zeros((BATCH, ACTIONS))
                for i in range(0, BATCH):
                    state_t = minibatch[i][0]
                    action_t = minibatch[i][1]
                    reward_t = minibatch[i][2]
                    state_t1 = minibatch[i][3]
                    done_t = minibatch[i][4]

                    inputs[i] = state_t
                    targets[i] = model.predict(state_t)
                    Q_sa = model.predict(state_t1)
                    if done_t:
                        #print('targets before done', targets)
                        targets[i, action_t] = reward_t
                        #print('targets after done', targets)
                    else:
                        targets[i, action_t] = reward_t + GAMMA * np.max(Q_sa[0])#[action_t]
                    if reward_t != 0.0:
                        ct_non_zero_reward += 1
                        #print('got reward', reward_t, 'for action', action_t)
                        #print('bef', model.predict(state_t))
                        #print('pred_s1', model.predict(state_t1))
                        #print('aft', targets)
                loss += model.train_on_batch(inputs, targets)
            rAll += r_t
            s_t = s_t1
            
            if done == True:
                break
            
        rewards.append(rAll)
        
        print('episode', idxEpisode, 'length', j, 'reward', rAll, 'epsilon', epsilon, 'loss sum', loss, 'non zero rewards', ct_non_zero_reward)
        
        if idxEpisode % 10 == 0:
            path = SAVE_DIR + 'model_episode_' + str(idxEpisode) + '.h5'
            save_model(model, path)

In [12]:
model = build_model()

We finish building the model


In [13]:
train_model(model, env)

episode 0 length 200 reward -4.0 epsilon 0.4 loss sum 0.0 non zero rewards 0
episode 1 length 200 reward -4.0 epsilon 0.4 loss sum 0.0 non zero rewards 0
episode 2 length 200 reward -4.0 epsilon 0.39996099999999835 loss sum 0.329763109341 non zero rewards 128
episode 3 length 200 reward -4.0 epsilon 0.399882999999995 loss sum 0.68240244515 non zero rewards 267
episode 4 length 200 reward -4.0 epsilon 0.39980499999999164 loss sum 0.677834153357 non zero rewards 266
episode 5 length 200 reward -1.0 epsilon 0.3997269999999883 loss sum 0.612662586936 non zero rewards 240
episode 6 length 200 reward -4.0 epsilon 0.39964899999998493 loss sum 0.583608548099 non zero rewards 228
episode 7 length 200 reward -3.0 epsilon 0.3995709999999816 loss sum 0.544505884402 non zero rewards 213
episode 8 length 200 reward -4.0 epsilon 0.3994929999999782 loss sum 0.477638290347 non zero rewards 187
episode 9 length 200 reward -4.0 epsilon 0.3994149999999749 loss sum 0.544214047281 non zero rewards 213
episo

episode 78 length 200 reward -4.0 epsilon 0.3940329999997435 loss sum 0.619068484386 non zero rewards 251
episode 79 length 200 reward -4.0 epsilon 0.39395499999974015 loss sum 0.666072935892 non zero rewards 271
episode 80 length 200 reward -4.0 epsilon 0.3938769999997368 loss sum 0.601094854024 non zero rewards 245
episode 81 length 200 reward -4.0 epsilon 0.39379899999973345 loss sum 0.601377297447 non zero rewards 244
episode 82 length 200 reward -1.0 epsilon 0.3937209999997301 loss sum 0.598746380485 non zero rewards 245
episode 83 length 200 reward -4.0 epsilon 0.39364299999972674 loss sum 0.598293859039 non zero rewards 244
episode 84 length 200 reward -4.0 epsilon 0.3935649999997234 loss sum 0.525069752675 non zero rewards 215
episode 85 length 200 reward -4.0 epsilon 0.39348699999972003 loss sum 0.522126669144 non zero rewards 214
episode 86 length 200 reward -4.0 epsilon 0.3934089999997167 loss sum 0.529133331063 non zero rewards 217
episode 87 length 200 reward -4.0 epsilon 

episode 155 length 200 reward -4.0 epsilon 0.3880269999994853 loss sum 0.310494869744 non zero rewards 290
episode 156 length 200 reward -4.0 epsilon 0.38794899999948196 loss sum 0.310831894545 non zero rewards 295
episode 157 length 200 reward -4.0 epsilon 0.3878709999994786 loss sum 0.326025641087 non zero rewards 284
episode 158 length 200 reward -4.0 epsilon 0.38779299999947525 loss sum 0.317522076162 non zero rewards 313
episode 159 length 200 reward -2.0 epsilon 0.3877149999994719 loss sum 0.292872585269 non zero rewards 270
episode 160 length 200 reward -3.0 epsilon 0.38763699999946855 loss sum 0.318978163617 non zero rewards 299
episode 161 length 200 reward -3.0 epsilon 0.3875589999994652 loss sum 0.313996095443 non zero rewards 304
episode 162 length 200 reward -3.0 epsilon 0.38748099999946184 loss sum 0.306651573366 non zero rewards 319
episode 163 length 200 reward -4.0 epsilon 0.3874029999994585 loss sum 0.335419663927 non zero rewards 324
episode 164 length 200 reward -4.

episode 232 length 200 reward -3.0 epsilon 0.3820209999992271 loss sum 0.247364137496 non zero rewards 374
episode 233 length 200 reward -4.0 epsilon 0.38194299999922376 loss sum 0.261426639205 non zero rewards 371
episode 234 length 200 reward -2.0 epsilon 0.3818649999992204 loss sum 0.272146479794 non zero rewards 347
episode 235 length 200 reward -4.0 epsilon 0.38178699999921706 loss sum 0.266144486188 non zero rewards 383
episode 236 length 200 reward -1.0 epsilon 0.3817089999992137 loss sum 0.257436216823 non zero rewards 345
episode 237 length 200 reward -2.0 epsilon 0.38163099999921035 loss sum 0.281484737177 non zero rewards 342
episode 238 length 200 reward -3.0 epsilon 0.381552999999207 loss sum 0.248547758616 non zero rewards 324
episode 239 length 200 reward -4.0 epsilon 0.38147499999920365 loss sum 0.264263461489 non zero rewards 336
episode 240 length 200 reward -3.0 epsilon 0.3813969999992003 loss sum 0.263472614301 non zero rewards 353
episode 241 length 200 reward -3.0

episode 309 length 200 reward -3.0 epsilon 0.3760149999989689 loss sum 0.228736663717 non zero rewards 376
episode 310 length 200 reward -3.0 epsilon 0.37593699999896557 loss sum 0.245727877889 non zero rewards 399
episode 311 length 200 reward -1.0 epsilon 0.3758589999989622 loss sum 0.233672142786 non zero rewards 402
episode 312 length 200 reward -2.0 epsilon 0.37578099999895886 loss sum 0.226367438561 non zero rewards 393
episode 313 length 200 reward -2.0 epsilon 0.3757029999989555 loss sum 0.201150887588 non zero rewards 365
episode 314 length 200 reward -3.0 epsilon 0.37562499999895216 loss sum 0.202067956474 non zero rewards 350
episode 315 length 200 reward -4.0 epsilon 0.3755469999989488 loss sum 0.250969744404 non zero rewards 412
episode 316 length 200 reward -1.0 epsilon 0.37546899999894545 loss sum 0.221060502401 non zero rewards 378
episode 317 length 200 reward -2.0 epsilon 0.3753909999989421 loss sum 0.220701324753 non zero rewards 384
episode 318 length 200 reward -4.

...

episode 1686 length 200 reward -2.0 epsilon 0.2686089999943516 loss sum 0.164894886388 non zero rewards 229
episode 1687 length 200 reward -2.0 epsilon 0.26853099999434826 loss sum 0.142232706959 non zero rewards 236
episode 1688 length 200 reward 0.0 epsilon 0.2684529999943449 loss sum 0.143156646096 non zero rewards 217
episode 1689 length 200 reward -1.0 epsilon 0.26837499999434156 loss sum 0.158735714445 non zero rewards 237
episode 1690 length 200 reward -2.0 epsilon 0.2682969999943382 loss sum 0.154236751609 non zero rewards 287
episode 1691 length 200 reward -3.0 epsilon 0.26821899999433485 loss sum 0.141524500301 non zero rewards 245
episode 1692 length 200 reward -1.0 epsilon 0.2681409999943315 loss sum 0.156112936689 non zero rewards 269
episode 1693 length 200 reward -2.0 epsilon 0.26806299999432814 loss sum 0.154702609958 non zero rewards 284
episode 1694 length 200 reward -2.0 epsilon 0.2679849999943248 loss sum 0.163984018931 non zero rewards 239
episode 1695 length 200 r

episode 1762 length 200 reward 0.0 epsilon 0.2626809999940968 loss sum 0.169657323248 non zero rewards 251
episode 1763 length 200 reward -2.0 epsilon 0.2626029999940934 loss sum 0.18466563776 non zero rewards 252
episode 1764 length 200 reward -2.0 epsilon 0.26252499999409007 loss sum 0.172439082933 non zero rewards 241
episode 1765 length 200 reward -1.0 epsilon 0.2624469999940867 loss sum 0.172864446591 non zero rewards 266
episode 1766 length 200 reward 1.0 epsilon 0.26236899999408336 loss sum 0.174561970227 non zero rewards 251
episode 1767 length 200 reward -1.0 epsilon 0.26229099999408 loss sum 0.169628542964 non zero rewards 244
episode 1768 length 200 reward -3.0 epsilon 0.26221299999407666 loss sum 0.17099462336 non zero rewards 234
episode 1769 length 200 reward -2.0 epsilon 0.2621349999940733 loss sum 0.176280973363 non zero rewards 249
episode 1770 length 200 reward -1.0 epsilon 0.26205699999406995 loss sum 0.175244777871 non zero rewards 245
episode 1771 length 200 reward

episode 1838 length 200 reward -1.0 epsilon 0.25675299999384193 loss sum 0.196701685461 non zero rewards 229
episode 1839 length 200 reward -4.0 epsilon 0.2566749999938386 loss sum 0.1926684879 non zero rewards 231
episode 1840 length 200 reward -1.0 epsilon 0.25659699999383523 loss sum 0.190569882892 non zero rewards 250
episode 1841 length 200 reward -2.0 epsilon 0.2565189999938319 loss sum 0.197473056061 non zero rewards 249
episode 1842 length 200 reward -1.0 epsilon 0.2564409999938285 loss sum 0.19906210748 non zero rewards 232
episode 1843 length 200 reward 2.0 epsilon 0.25636299999382517 loss sum 0.205770074477 non zero rewards 238
episode 1844 length 200 reward -2.0 epsilon 0.2562849999938218 loss sum 0.197956076096 non zero rewards 244
episode 1845 length 200 reward 0.0 epsilon 0.25620699999381846 loss sum 0.217469363502 non zero rewards 237
episode 1846 length 200 reward 0.0 epsilon 0.2561289999938151 loss sum 0.206094524052 non zero rewards 215
episode 1847 length 200 reward

episode 1914 length 200 reward -2.0 epsilon 0.2508249999935871 loss sum 0.197807475342 non zero rewards 227
episode 1915 length 200 reward -1.0 epsilon 0.25074699999358374 loss sum 0.20661284955 non zero rewards 245
episode 1916 length 200 reward -4.0 epsilon 0.2506689999935804 loss sum 0.193704183213 non zero rewards 226
episode 1917 length 200 reward -1.0 epsilon 0.25059099999357703 loss sum nan non zero rewards 258
episode 1918 length 200 reward -3.0 epsilon 0.2505129999935737 loss sum nan non zero rewards 228
episode 1919 length 200 reward -4.0 epsilon 0.25043499999357033 loss sum nan non zero rewards 244
episode 1920 length 200 reward -4.0 epsilon 0.250356999993567 loss sum nan non zero rewards 234
episode 1921 length 200 reward -4.0 epsilon 0.2502789999935636 loss sum nan non zero rewards 268
episode 1922 length 200 reward -3.0 epsilon 0.25020099999356027 loss sum nan non zero rewards 273
episode 1923 length 200 reward 0.0 epsilon 0.2501229999935569 loss sum nan non zero rewards 

episode 1998 length 200 reward -2.0 epsilon 0.24427299999371302 loss sum nan non zero rewards 355
episode 1999 length 200 reward -4.0 epsilon 0.24419499999371522 loss sum nan non zero rewards 332


In [14]:
from matplotlib import pyplot as plt
plt.plot(range(len(rewards)), rewards)

NameError: name 'rewards' is not defined

test_model(model, Wrapped_Game(env))